In [ ]:
headers = {"User-Agent": ua.random}
def get_driver():
    """This function is used to set up a Selenium WebDriver with headless Chrome 
    in order to go through the data within Yelp."""
    options = Options()
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")       
    options.add_argument("--window-size=1920,1080")
    time.sleep(5)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver
driver = get_driver()

def community_section_len(driver):
    try:
        ask_community_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//section[contains(@aria-label, 'Ask the Community')]"))
        )
        community_section = []
        for i in ask_community_section:
            try:
                community_dates = ask_community_section.find_elements(By.XPATH, ".//span[contains(@class, 'y-css-1d8mpv1')]")
                if community_dates:
                    community_section.append(community_dates)
                else: 
                    continue
            except Exception as e:
                print(f"Error extracting rating: {e}") 
        length = len(community_section)
        return length
    
    except TimeoutException:
        print("Ask the Community section not found.")
        return 0

community_section_len(driver)

In [ ]:
def get_driver():
    """This function is used to set up a Selenium WebDriver with headless Chrome 
    in order to go through the data within Yelp."""
    options = Options()
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")       
    options.add_argument("--window-size=1920,1080")
    time.sleep(5)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def yelp_dates(clean_yelp_url):
    "This function extracts the time at which each yelp rating was posted"
    driver = get_driver()
    driver.get(clean_yelp_url)
    time.sleep(5) # <-- allows time for javascript to load
    ask_community_reviews = driver.find_elements(By.XPATH, "//section[@aria-label='Ask the Community']//ul[contains(@class, 'list__09f24__ynIEd')]/li")
    length = len(ask_community_reviews)
    all_page_dates = []
    page = 1
    while True:
        time.sleep(3)
        review_sections = driver.find_elements(By.XPATH, "//ul[contains(@class, 'list__09f24__ynIEd')]//li") # <--- tells the webscraper to start scraping at the start constumer reviews section
        print(f"Found {len(review_sections)} dates.")  # Debugging
        if not review_sections:
            print("No dates found on this page.")
            break 
        
        dates = []
        for d in review_sections:
            try:
                date_element = d.find_element(By.XPATH, ".//span[contains(@class, 'y-css-1d8mpv1')][not(contains(text(), 'Photos'))]")
                date = date_element.text.strip()  
                if date:
                    dates.append(date)
                else:
                    continue
            except Exception as e:
                print(f"Error extracting rating: {e}") 
        all_page_dates.append(dates)
        print(f"Stored {len(dates)} from Page {page} into all_page_dates")
        if page > 2:
            print(f"Reached page limit {page}")
            break
        if not click_next_page(driver):
            print("No more pages to scrape reviews from")
            break
        page += 1
    driver.quit()
    return all_page_dates
    
all_dates = yelp_dates(clean_yelp_url)
print(all_dates)

In [ ]:
def yelp_ratings(clean_yelp_url):
    "This function extracts each yelp rating using Selenium"
    driver = get_driver()
    driver.get(clean_yelp_url)
    time.sleep(5) # <-- allows time for javascript to load
    all_page_ratings = []
    page = 1
    while True:
        time.sleep(2)
        review_sections = driver.find_elements(By.XPATH, "//ul[contains(@class, 'list__09f24__ynIEd')]//li") # <--- tells the webscraper to start scraping at the start constumer reviews section
        print(f"Found {len(review_sections)} reviews.")  # Debugging
        if not review_sections:
            print("No reviews found on this page.")
            break 
        
        ratings = []
        for review in review_sections:
            try:
                rating_element = review.find_elements(By.XPATH, ".//div[contains(@class, 'y-css-dnttlc')]")  
                if rating_element:
                    rating = rating_element[0].get_attribute("aria-label") 
                    ratings.append(rating)
                else:
                    continue
            except Exception as e:
                print(f"Error extracting rating: {e}") 
        time.sleep(2)
        all_page_ratings.append(ratings)
        print(f"Stored {len(ratings)} from Page {page} into all_page_ratings")
        if page > 2:
            print(f"Reached page limit {page}")
            break
        if not click_next_page(driver):
            print("No more pages to scrape reviews from")
            break
        page += 1
    driver.quit()
    return all_page_ratings

all_reviews = yelp_ratings(clean_yelp_url)
print(all_reviews)